<a href="https://colab.research.google.com/github/Chandrashekar0123/Age-and-Gender-Prediction/blob/main/Age_and_Gender_Detection_GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Install all necessary packages


In [1]:
pip install flask pandas scikit-learn

In [2]:
!pip install pyngrok==4.1.1

In [3]:
!pip install flask_ngrok

# Authenticate Ngrok with your token
!ngrok authtoken 2rwq2V6fDFhbDZDmrx5mxxwhBwo_4wPq1yP8imGyUA4Z1xPN4

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!pip install requests
!pip install pyngrok


^C


In [4]:
pip install --upgrade pyngrok

  Using cached pyngrok-7.2.3-py3-none-any.whl.metadata (8.7 kB)
Using cached pyngrok-7.2.3-py3-none-any.whl (23 kB)
  Attempting uninstall: pyngrok
    Found existing installation: pyngrok 4.1.1
    Uninstalling pyngrok-4.1.1:
      Successfully uninstalled pyngrok-4.1.1


In [5]:
from pyngrok import ngrok

# Replace 'your_authtoken_here' with your actual ngrok authtoken
ngrok.set_auth_token("2rwq2V6fDqwedfdssddFhbDZDmrx5mxfdwsdfdxwhBwo_4wPq1yP8imGyUA4Z1xPN4")


In [6]:

!wget -O ngrok.zip https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-stable-linux-amd64.zip
!unzip ngrok.zip
!mv ngrok /usr/local/bin/


--2025-02-15 17:03:06--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 99.83.220.108, 13.248.244.96, 35.71.179.82, ...
Connecting to bin.equinox.io (bin.equinox.io)|99.83.220.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14949170 (14M) [application/octet-stream]
Saving to: ‘ngrok.zip’

ngrok.zip           100%[===================>]  14.26M  17.8MB/s    in 0.8s    

2025-02-15 17:03:09 (17.8 MB/s) - ‘ngrok.zip’ saved [14949170/14949170]

Archive:  ngrok.zip
  inflating: ngrok                   


In [7]:
!ngrok authtoken 2rwq2V6fDFhbDZDwsdwdwsdmrx5mxxwhBwsdfwsdo_4wPq1yP8imGyUA4Z1xPN4


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pkill -f ngrok


In [ ]:
!rm -rf /root/.ngrok2/ngrok.yml
!rm -rf /root/.config/ngrok/ngrok.yml


In [ ]:
!ngrok authtoken 2rwq2V6fDFhbDZDmrx5mqwqwdsxxwhBwo_4wPq1yP8imGyUdwedwedA4Z1xPN4


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:
from pyngrok import ngrok
public_url = ngrok.connect(5000)
print(f"Flask app is accessible at: {public_url}")


Flask app is accessible at: NgrokTunnel: "https://3eaf-34-19-81-155.ngrok-free.app" -> "http://localhost:5000"


##Python Code and GUI

In [11]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from flask import Flask, request, render_template_string, redirect, url_for
from PIL import Image
import io
import os
import base64
from pyngrok import ngrok

# Load the trained model
MODEL_PATH = "/content/Age_Sex_Detection.keras"  # Update if needed

if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Model file not found at: {MODEL_PATH}")

model = load_model(MODEL_PATH)

# Initialize Flask app
app = Flask(__name__)

# Set up ngrok for public URL
ngrok.set_auth_token("2rwq2V6fDFhbDZDmrx5mxxwhBwasaqsqsdo_4wPq1yP8imGyUA4Z1sdfgfdxPN4")  # Replace with your actual token
public_url = ngrok.connect(5000).public_url
print(f"Flask app is accessible at: {public_url}")

# HTML Templates
UPLOAD_PAGE = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Upload Image | Age & Gender Detector</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
    <style>
        body { text-align: center; background: #F0F8FF; padding: 50px; }
        .container { max-width: 500px; background: white; padding: 20px; border-radius: 10px; box-shadow: 0px 4px 10px rgba(0,0,0,0.1); }
    </style>
</head>
<body>
    <div class="container">
        <h2 class="mb-4 text-primary">Upload an Image</h2>
        <form action="/predict" method="post" enctype="multipart/form-data">
            <input type="file" name="file" class="form-control mb-3" required>
            <button type="submit" class="btn btn-primary">Predict</button>
        </form>
    </div>
</body>
</html>
'''

RESULT_PAGE = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Prediction Results</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
    <style>
        body { text-align: center; background: #E6F7FF; padding: 50px; }
        .container { max-width: 600px; background: white; padding: 20px; border-radius: 10px; box-shadow: 0px 4px 10px rgba(0,0,0,0.1); }
        img { max-width: 100%; border-radius: 10px; margin-top: 10px; }
    </style>
</head>
<body>
    <div class="container">
        <h2 class="text-success">Prediction Results</h2>
        <img src="data:image/jpeg;base64,{{ image_data }}" alt="Uploaded Image">
        <h4 class="mt-3">Predicted Age: <strong>{{ age }}</strong></h4>
        <h4>Predicted Gender: <strong>{{ gender }}</strong></h4>
        <a href="/" class="btn btn-primary mt-3">Upload Another Image</a>
    </div>
</body>
</html>
'''

# Function to preprocess image and make prediction
def detect_age_gender(image):
    image = image.convert("RGB")  # Convert to RGB
    image = image.resize((48, 48))  # Resize to match model input
    image = np.array(image) / 255.0  # Normalize
    image = image[:, :, :3]  # Ensure only 3 channels (RGB)
    image = np.expand_dims(image, axis=0)  # Expand dims for batch

    sex_labels = ["Male", "Female"]
    pred = model.predict(image)

    age = int(np.round(pred[1][0]))  # Predicted age
    gender = sex_labels[int(np.round(pred[0][0]))]  # Predicted gender

    return age, gender

# Flask Routes
@app.route('/')
def home():
    return render_template_string(UPLOAD_PAGE)  # Upload Page

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return redirect(url_for('home'))

    file = request.files['file']
    if file:
        image = Image.open(io.BytesIO(file.read()))
        age, gender = detect_age_gender(image)

        # Convert image to base64 for displaying
        img_io = io.BytesIO()
        image.save(img_io, format="JPEG")
        img_io.seek(0)
        img_base64 = base64.b64encode(img_io.read()).decode("utf-8")

        return render_template_string(RESULT_PAGE, age=age, gender=gender, image_data=img_base64)

    return redirect(url_for('home'))

# Start Flask App
if __name__ == '__main__':
    app.run(port=5000)


Flask app is accessible at: https://fd13-34-19-81-155.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Feb/2025 17:16:54] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Feb/2025 17:16:55] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 736ms/step


<ipython-input-11-f60ab790e7e8>:91: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  age = int(np.round(pred[1][0]))  # Predicted age
<ipython-input-11-f60ab790e7e8>:92: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  gender = sex_labels[int(np.round(pred[0][0]))]  # Predicted gender
INFO:werkzeug:127.0.0.1 - - [15/Feb/2025 17:17:07] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Feb/2025 17:17:18] "GET / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step


...

INFO:werkzeug:127.0.0.1 - - [15/Feb/2025 17:18:51] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Feb/2025 17:19:22] "GET / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step


INFO:werkzeug:127.0.0.1 - - [15/Feb/2025 17:19:45] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Feb/2025 17:20:07] "GET / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step


INFO:werkzeug:127.0.0.1 - - [15/Feb/2025 17:20:22] "POST /predict HTTP/1.1" 200 -
